In [106]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/testing/Test.xlsx
/kaggle/input/train-data/Train.xlsx


In [267]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_excel('/kaggle/input/train-data/Train.xlsx',header=0)
df

,NO,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q69,Q70,Q71,Q72,Q73,Q74,Q75,Q76,Q77,CLASS
0,11,1,5,2,5,1,3,1,1,1,...,4,3,1,4,1,2,2,3,4,1
1,12,2,5,1,5,1,5,1,1,1,...,5,3,1,5,1,2,1,3,1,1
2,13,2,4,2,5,2,5,2,2,2,...,4,3,2,3,2,3,2,2,4,1
3,14,2,5,1,5,2,5,2,1,2,...,3,2,2,4,2,2,4,3,4,1
4,15,1,4,1,5,1,3,1,1,1,...,4,2,2,3,1,3,1,3,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,297,3,4,3,5,4,5,2,2,2,...,5,1,3,4,1,3,4,4,4,3
277,298,3,4,3,5,2,3,3,1,2,...,2,3,4,3,3,3,4,3,3,3
278,299,2,4,3,3,2,4,2,1,1,...,5,4,2,4,2,3,3,3,4,3
279,300,4,4,1,5,3,2,2,2,2,...,4,3,3,4,3,3,4,4,4,3


In [268]:
df.shape

(281, 79)

In [269]:
df.isnull().sum(axis=0)

NO       0
Q1       0
Q2       0
Q3       0
Q4       0
        ..
Q74      0
Q75      0
Q76      0
Q77      0
CLASS    0
Length: 79, dtype: int64

In [270]:
df_clean = df.dropna(0,'any')

In [271]:
np.shape(df_clean)

(280, 79)

In [272]:
pel_class = np.unique(df_clean['CLASS'])
pel_class

array([1, 2, 3])

In [273]:
pel_value = [sum(df_clean['CLASS'] == pel_class[0]),
            sum(df_clean['CLASS'] == pel_class[1]),
            sum(df_clean['CLASS'] == pel_class[2])]
pel_value

[134, 85, 61]

In [274]:
y = df_clean['CLASS'].values
X = df_clean.drop(['CLASS'], axis = 1).values

In [275]:
np.shape(y)

(280,)

In [276]:
df_clean.shape

(280, 79)

In [277]:
np.shape(X)

(280, 78)

In [278]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X, y = sm.fit_resample(X,y)

In [279]:
sum(y == 3)

134

In [301]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)


from sklearn.decomposition import PCA
fr = PCA(n_components=1)
fr.fit(X_train)
X_train = fr.transform(X_train)
X_test = fr.transform(X_test)


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
    'n_estimators' : [10,50,100],
    'min_samples_leaf' : [1,2,4],
    'max_features' : ['sqrt',0.5,0.8],
}
clf = GridSearchCV(RandomForestClassifier(random_state=0),params, cv = 10)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

Best params : {'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 10}
10CV accuracy : 100.0


In [302]:
np.shape(X_train)

(201, 1)

In [303]:
df = pd.read_excel('/kaggle/input/testing/Test.xlsx',header=0)

In [294]:
yt = df['CLASS'].values
Xt = df.drop(['CLASS'], axis = 1).values

In [295]:
Xt = fr.transform(Xt)

In [296]:
X_test_norm = scaler.transform(Xt)

In [297]:
np.shape(X_test_norm)

(30, 1)

In [298]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = clf.predict(X_test_norm)

In [299]:
y_predict

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3])

In [300]:
print("Test accuracy : "+str(sum(yt == y_predict)/len(yt)*100))

Test accuracy : 80.0


In [186]:
X_test_norm

array([[-0.03459384,  0.19610293,  0.28126667,  0.21754702,  0.35997413,
         0.45261407,  0.22438162,  0.58493764,  0.11738767,  0.52074447],
       [-0.03115038,  0.2117872 ,  0.36577608,  0.31049073,  0.43780375,
         0.60044774,  0.29272264,  0.48863666,  0.41468221,  0.45954197],
       [-0.02768118,  0.22949048,  0.44011851,  0.13536754,  0.66390338,
         0.52068661,  0.21870177,  0.63580632,  0.79831387,  0.62833895],
       [-0.02421838,  0.17270895,  0.37755138,  0.2160935 ,  0.45538064,
         0.64433003,  0.24989177,  0.444362  ,  0.31494281,  0.53806404],
       [-0.02081819,  0.21378059,  0.40593973,  0.81840144,  0.36092967,
         0.39285994,  0.27057598,  0.46195906,  0.46131437,  0.38628723],
       [-0.01724099,  0.28568367,  0.57113356,  0.40753527,  0.48352411,
         0.46646706,  0.22045994,  0.31715857,  0.57533906,  0.46496433],
       [-0.01379249,  0.25348925,  0.37466985,  0.21722643,  0.45286207,
         0.63997037,  0.15421502,  0.29060663